1. Location : A small subsection of New York Central, with Water Bodis
2. Motivation :  I have never been to New York, but its the subject of many movies. I wanted to understand the topography, and see it through the eyes of an analyst
3. DataSet : ny_extract_osm1.xml [link](link)

You will choose any area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of the world that you care about. Finally, you will choose either MongoDB or SQL as the data schema to complete your project.

1. Udacity "Data Wrangling Using Mongo DB" (Lesson 6)
2. Mongo DB Docs 

1. Data Download
2. Data Audit
3. Data Cleansing/Problem Resolution
4. Load Data into MongoDB and Insights 
5. Conclusion

I can extract using the Overpass API using the Co-ordinates that are specified in the code below

In [2]:
"""I am doing this using the  Browser, and so have commented out this code"""
import urllib
#for the Overpass API Git
#file00 = urllib.URLopener()
#file00.retrieve("http://overpass-api.de/api/map? bbox=77.3901,12.8620,77.7973,13.0815", "P3 - Data Wrangling using NoSQL/new-york_new-york.osm/ny_extract_osm.xml""

I can also download the predefined extract using MapZen, which can be downloaded using a link
[https://mapzen.com/data/metro-extracts/metro/bengaluru_india](https://mapzen.com/data/metro-extracts/metro/bengaluru_india/)
The downloaded data is available in this project itself

In [2]:
#Imports Master

import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections
import pymongo
from pymongo import MongoClient
import os

In [21]:
#File Master

#osm_file = "P3 - Data Wrangling using NoSQL/bengaluru_india.osm/bengaluru_india.osm"
osm_file = "P3 - Data Wrangling using NoSQL/new-york_new-york.osm/ny_extract_osm1.xml"
os.path.exists(osm_file)

True

In [6]:
print 'The size of the OSM file is {}'.format(os.path.getsize(osm_file)/1.0e6)

The size of the OSM file is 511.26328


In [7]:
#Using Lesson 6 Will count the no of tags

def count_tags(filename):
    tags = {}
    print filename
    for event,elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags
count_tags(osm_file)

P3 - Data Wrangling using NoSQL/new-york_new-york.osm/ny_extract_osm1.xml


{'bounds': 1,
 'member': 97766,
 'meta': 1,
 'nd': 2660357,
 'node': 1925728,
 'note': 1,
 'osm': 1,
 'relation': 2887,
 'tag': 1893802,
 'way': 346018}

In [8]:
#Print a sample of the OSM File
#Code sample based on Quiz of Data Wrangling using MongoDB

# Print a sample of the file. Here I am printing the first 100 elements

OSM_FILE = osm_file  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 100 # Parameter: print the first k elements (or every kth element)

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


# print the first  k top level elements
for i, element in enumerate(get_element(OSM_FILE)):
    if i <= k:
        print ET.tostring(element)
'''
# or print every  kth top level elements
for i, element in enumerate(get_element(OSM_FILE)):
    if i % k == 0:
        print ET.tostring(element)
'''
# Also writing the sample file to view it in Sublime text
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

<node changeset="41015803" id="26769789" lat="40.6995927" lon="-74.1868914" timestamp="2016-07-25T17:17:46Z" uid="326503" user="wambag" version="4" />
  
<node changeset="40698552" id="26769792" lat="40.6962016" lon="-74.1779077" timestamp="2016-07-12T22:13:28Z" uid="326503" user="wambag" version="5" />
  
<node changeset="41096881" id="26769800" lat="40.6858690" lon="-74.1908483" timestamp="2016-07-28T23:41:20Z" uid="326503" user="wambag" version="5">
    <tag k="railway" v="switch" />
  </node>
  
<node changeset="41096881" id="26769802" lat="40.6866906" lon="-74.1907921" timestamp="2016-07-28T23:41:20Z" uid="326503" user="wambag" version="5" />
  
<node changeset="41024408" id="26769831" lat="40.6919659" lon="-74.1761197" timestamp="2016-07-25T22:35:09Z" uid="326503" user="wambag" version="5" />
  
<node changeset="41024408" id="26769832" lat="40.6927831" lon="-74.1759395" timestamp="2016-07-25T22:35:09Z" uid="326503" user="wambag" version="5" />
  
<node changeset="41024408" id="26

We will iterate over all the data, and for any K, V we will check if the value has only lower, or lower with a : or any other characters. This will help us determine if any data cleansing is required for this data

We will take a count of this, which we will display below

import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        for val in element.iter('tag'):
            k = element.get('k')
            #search for the "lower"
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [9]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        for val in element.iter('tag'):
            k = element.get('k')
            #search for the "lower"
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [10]:
sc_tag_category = process_map(osm_file)
pprint.pprint(sc_tag_category)

{'lower': 764249,
 'lower_colon': 1100117,
 'other': 12111,
 'problemchars': 17325}


We will get a count of the users, who have updated. This will give us an idea, of how active this section is, and how much variation we can expect on the data.

In [11]:
# this function will tell us how many unique users have already contributed to the map data

def get_user(element):
    if element.get('uid'):
        uid = element.attrib['uid']
        return uid
    else:
        return None
    
    

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))

    return users

contributing_users = process_map(osm_file)
len(contributing_users)

2174

So 2174 users have contributed to the map data. This is a good number to start.

This is local data set, from the Asia Region. So the standard correction algorithm will not work. First we will find the various street names, based on this we will decide the approach to eliminate variation.

In [16]:
from collections import defaultdict
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = {}

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def get_street_names(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    value = tag.attrib['v']
                    m = street_type_re.search(value)
                    if m:
                        street_type = m.group()
                        street_types[street_type].add(value)
                        

    return street_types

street_types = get_street_names(osm_file)
street_types.keys()

['Heights',
 'Rd',
 'Turnpike',
 'avenue',
 'M204',
 'Southwest',
 'Mews',
 '27th',
 'Landing',
 'Willoughby',
 '403',
 '400',
 'D',
 '2R',
 'Level',
 'H',
 'L',
 'P',
 '2N',
 'Piers',
 'Finest',
 'Alley',
 'bus_stop',
 'Broadway',
 'Warren',
 '29th',
 'West',
 'Rico',
 'street',
 'M1',
 'E',
 'Circle',
 'East',
 'Unidos',
 'Park',
 'Plz',
 'Remsen',
 '3',
 'Yards',
 '301',
 'C',
 'Ave.',
 'Extension',
 '42nd',
 'K',
 'Plaza',
 '306',
 'Oval',
 'Mall',
 'Place',
 '11217',
 'Loop',
 'Streeet',
 'Bushwick',
 'Highline',
 'Square',
 'Clinton',
 'S',
 'N',
 'Floor)',
 'Row',
 'Boulevard',
 'St.',
 'Way',
 '1st',
 '4CF',
 'North',
 'Macdougal',
 'Vanderbilt',
 'NY',
 'Brooklyn',
 'Fulton',
 'Walkway',
 '506',
 '507',
 '500',
 'Roadbed',
 '1807',
 'Lane',
 'B',
 'Center',
 'F',
 'NJ)',
 'J',
 '900',
 'St',
 'Steet',
 'Parkway',
 'Road',
 '861',
 'Esplanade',
 'Walk',
 'Atrium',
 'Ave',
 'Avenue',
 'Court',
 'USA',
 'Floor',
 '1204',
 'Hudson',
 '800a',
 'Terminal',
 '633',
 'ave',
 'Highway'

We can now define a prelimnary mapping file. Do note that, there are a number of non standard names in the street names list. These will not be corrected, but the mapping file will work for all the standard names, where there are variations

In [14]:
mapping = {'Ave'  : 'Avenue',
           'ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'ST'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square"}

In [20]:
def update_street_name(name, mapping):

    m = street_type_re.search(name)
    better_name = name
    
    if m:
        if m.group() in mapping.keys():
            better_street_type = mapping[m.group()]
            better_name = street_type_re.sub(better_street_type,name)
            
    return better_name

print '**************************MODIFIED STREET NAMES**************************'
for street_type, ways in street_types.iteritems():
    for name in ways:
        better_name = update_street_name(name, mapping)
        print name, "=>", better_name

**************************MODIFIED STREET NAMES**************************
Columbia Heights => Columbia Heights
Albemarle Rd => Albemarle Road
43rd Rd => 43rd Road
Newark Turnpike => Newark Turnpike
seventh avenue => seventh avenue
Bedford avenue => Bedford avenue
2nd avenue => 2nd avenue
Utica avenue => Utica avenue
3rd Ave #M204 => 3rd Ave #M204
Prospect Park Southwest => Prospect Park Southwest
Greenwich Mews => Greenwich Mews
Washington Mews => Washington Mews
W 27th => W 27th
Bay Street Landing => Bay Street Landing
Willoughby => Willoughby
East 25th Street  #403 => East 25th Street  #403
East 25th Street Ste 400 => East 25th Street Ste 400
Avenue D => Avenue D
408 West 22nd Street #2R => 408 West 22nd Street #2R
Madison Ave Arcage Level => Madison Ave Arcage Level
Avenue H => Avenue H
Avenue L => Avenue L
Avenue P => Avenue P
400th West 20th St., Suite 2N => 400th West 20th St., Suite 2N
Northside Piers => Northside Piers
Avenue Of The Finest => Avenue Of The Finest
Exchange Alley

Cambridge Place
Garden Place => Garden Place
Hampton Place => Hampton Place
Ridgewood Place => Ridgewood Place
La Grange Place => La Grange Place
Visitation Place => Visitation Place
Saint James Place => Saint James Place
Emerson Place => Emerson Place
Rutherford Place => Rutherford Place
Forest Place => Forest Place
Austin Place => Austin Place
Warren Place => Warren Place
Bancroft Place => Bancroft Place
Federal Place => Federal Place
Oakland Place => Oakland Place
Turner Place => Turner Place
Mackay Place => Mackay Place
Irvington Place => Irvington Place
Astor Place => Astor Place
Hygeia Place => Hygeia Place
Eunice Place => Eunice Place
Boerum Place => Boerum Place
Elm Place => Elm Place
Reeve Place => Reeve Place
Dolson Place => Dolson Place
Vanderveer Place => Vanderveer Place
Brooks Place => Brooks Place
Sterling Place => Sterling Place
Strong Place => Strong Place
North 6th Place => North 6th Place
North Elliott Place => North Elliott Place
College Place => College Place
Divis

North Randall Avenue
Anderson Avenue => Anderson Avenue
Comstock Avenue => Comstock Avenue
Elizabeth Avenue => Elizabeth Avenue
Post Avenue => Post Avenue
Graham Avenue => Graham Avenue
Bement Avenue => Bement Avenue
Drake Avenue => Drake Avenue
Woodruff Avenue => Woodruff Avenue
Leyden Avenue => Leyden Avenue
Burden Avenue => Burden Avenue
Saint Nicholas Avenue => Saint Nicholas Avenue
Slosson Avenue => Slosson Avenue
Greenwood Avenue => Greenwood Avenue
Monroe Avenue => Monroe Avenue
57th Avenue => 57th Avenue
Wardwell Avenue => Wardwell Avenue
Onderdonk Avenue => Onderdonk Avenue
Parkside Avenue => Parkside Avenue
6th Avenue => 6th Avenue
Coney Island Avenue => Coney Island Avenue
Elmira Avenue => Elmira Avenue
Curtis Avenue => Curtis Avenue
Greenpoint Avenue => Greenpoint Avenue
Villa Avenue => Villa Avenue
Ronald Avenue => Ronald Avenue
Metropolitan Avenue => Metropolitan Avenue
Benziger Avenue => Benziger Avenue
Morgan Avenue => Morgan Avenue
Atlantic Avenue => Atlantic Avenue
Ma

=> Gerry Street
East 56th Street => East 56th Street
Waterbury Street => Waterbury Street
Chambers Street => Chambers Street
39th Street => 39th Street
Bush Street => Bush Street
Pearl Street => Pearl Street
Ludwig Street => Ludwig Street
Dongan Street => Dongan Street
Tapscott Street => Tapscott Street
George Street => George Street
Congress Street => Congress Street
Frost Street => Frost Street
Dyson Street => Dyson Street
37th Avenue, 14th Street, 21st Street => 37th Avenue, 14th Street, 21st Street
Grove Street => Grove Street
Bosworth Street => Bosworth Street
West 47th Street => West 47th Street
Nelson Street => Nelson Street
Howard Street => Howard Street
West 57th Street => West 57th Street
West 23rd Street => West 23rd Street
North 13th Street => North 13th Street
Rector Street => Rector Street
Story Street => Story Street
Winant Street => Winant Street
Plymouth Street => Plymouth Street
Ferris Street => Ferris Street
West Thames Street => West Thames Street
Dover Street => Do

First we will extract the pin codes, and based on the sampling, we will put in a logic to extract the errors. 

In [33]:
from collections import defaultdict

def list_zipcode(invalid_zipcode,zipcode, categorized_master_list_zipcodes):
    initial_digits = zipcode[0:2]   
    
    """
    Create an initial set of the expectation and find out outliers
    Ref : https://www.unitedstateszipcodes.org/ny/
            Shows list of valid zip codes for New York
    """
    if not initial_digits.isdigit():
        invalid_zipcode[initial_digits].add(zipcode)   
    
    elif int(initial_digits) not in [10,11,12,13,14]:
        invalid_zipcode[initial_digits].add(zipcode)
   
    categorized_master_list_zipcodes[initial_digits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def filter_invalid_zipcodes(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    categorized_master_list_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    list_zipcode(invalid_zipcodes,tag.attrib['v'], categorized_master_list_zipcodes)

    return invalid_zipcodes, categorized_master_list_zipcodes

invalid_zipcodes, categorized_master_list_zipcodes = filter_invalid_zipcodes(osm_file)

Most of the data looks correct, and some standardizations can be applied

1.	Remove the leading NY for all the valid 5 digit codes
2. 	Trim the data after the leading '-' or ';' character as it might be non standard information

There is some data which looks incorrect, like zip codes which start with NJ, or codes which start with 07. Since the data itself is incorrect, no standardizations can be applied

In [41]:
def cleanse_invalid_zip_codes(zipcode):           
    alpha = re.findall('[a-zA-Z]*', zipcode)
    if alpha:
        first_few_letters = alpha[0]
        
        if first_few_letters == 'NJ':
            #We wont correct NJ Data
            return zip_code
        else:
            converted_zip_code = re.findall(r'\d+',zip_code)            
            if converted_zip_code:
                if converted_zip_code.__len__() > 0:
                    #This is a - seperated code
                    return converted_zip_code[0]
                else:
                    return converted_zip_code
            else:
                #All is well
                return zip_code

for zip_category, zip_codes in categorized_master_list_zipcodes.iteritems():
    for zip_code in zip_codes:
        better_code = cleanse_invalid_zip_codes(zip_code)
        print zip_code, "=>", better_code

11201;11231 => 11201
11228 => 11228
11201 => 11201
11227 => 11227
11203 => 11203
11205 => 11205
11204 => 11204
11229 => 11229
11206 => 11206
11209 => 11209
11226 => 11226
11225 => 11225
11232 => 11232
11221 => 11221
11220 => 11220
11219 => 11219
11249 => 11249
11385 => 11385
11215-9993 => 11215
11232-2400 => 11232
11222 => 11222
11238 => 11238
11213 => 11213
11210 => 11210
11211 => 11211
11216 => 11216
11217 => 11217
11215 => 11215
11230 => 11230
11231 => 11231
11218 => 11218
11233 => 11233
11234 => 11234
11236 => 11236
11237 => 11237
11212 => 11212
11109 => 11109
11378 => 11378
11103 => 11103
11377 => 11377
11101 => 11101
11106 => 11106
11104 => 11104
11251 => 11251
11231;11230 => 11231
10004 => 10004
10005 => 10005
10006 => 10006
10007 => 10007
10001 => 10001
10002 => 10002
10003 => 10003
10001-2062 => 10001
10305 => 10305
10304 => 10304
10303 => 10303
10009 => 10009
10301 => 10301
10045 => 10045
10123 => 10123
10016-0122 => 10016
10022 => 10022
10121 => 10121
10020 => 10020
10018-45

For the purpose we can use the code that we prepared in the case study, where we had the below rules: 

-	you should process only 2 types of top level tags: "node" and "way"
-	all attributes of "node" and "way" should be turned into regular key/value pairs, except:
-	attributes in the CREATED array should be added under a key "created"
-	attributes for latitude and longitude should be added to a "pos" array, for use in geospacial indexing. Make sure the values inside "pos" array are floats and not strings.
-	if the second level tag "k" value contains problematic characters, it should be ignored
-	if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
-	if the second level tag "k" value does not start with "addr:", but contains ":", you can process it in a way that you feel is best. For example, you might split it into a two-level dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
-	if there is a second ":" that separates the type/direction of a street, the tag should be ignored

In [22]:
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way":
        # YOUR CODE HERE
        node['id'] = element.attrib['id']
        node['type'] = element.tag
        node['visible'] = element.get('visible')
        created = dict()
        created['version'] = element.attrib['version']
        created['changeset'] = element.attrib['changeset']
        created['timestamp'] = element.attrib['timestamp']
        created['user'] = element.attrib['user']
        created['uid'] = element.attrib['uid']
        node['created'] = created
        if "lat" in element.keys() and 'lon' in element.keys():
            pos = []
            pos.append(float(element.attrib['lat']))
            pos.append(float(element.attrib['lon']))
            node['pos'] = pos
        else:
            node['pos'] = None
        addr = {}
        for tag in element.iter("tag"):
            tag_name = tag.attrib['k']
        
            tag_value = tag.attrib['v']
            if problemchars.search(tag_name):
                #skip the problem chars
                continue
            if 'addr'in tag_name:
                add_key = tag_name.split(':',1)[1]
                if add_key in ('housenumber', 'street'):
                    addr[add_key] = tag_value
            elif tag_name == "name":
                node['name'] = tag_value
            elif tag_name == 'phone':
                node['phone'] = tag_value
            elif tag_name == 'amenity':
                node['amenity'] = tag_value
            elif tag_name == 'cuisine':
                node['cuisine'] = tag_value
            node['address'] = addr
        


        return node
    else:
        return None

In [23]:
def process_map(file_in, pretty=False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2) + "\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [24]:
#osm_file = 'sample.osm'
data = process_map(osm_file)

for i in range(10):
    pprint.pprint(data[i])

{'created': {'changeset': '41015803',
             'timestamp': '2016-07-25T17:17:46Z',
             'uid': '326503',
             'user': 'wambag',
             'version': '4'},
 'id': '26769789',
 'pos': [40.6995927, -74.1868914],
 'type': 'node',
 'visible': None}
{'created': {'changeset': '40698552',
             'timestamp': '2016-07-12T22:13:28Z',
             'uid': '326503',
             'user': 'wambag',
             'version': '5'},
 'id': '26769792',
 'pos': [40.6962016, -74.1779077],
 'type': 'node',
 'visible': None}
{'address': {},
 'created': {'changeset': '41096881',
             'timestamp': '2016-07-28T23:41:20Z',
             'uid': '326503',
             'user': 'wambag',
             'version': '5'},
 'id': '26769800',
 'pos': [40.685869, -74.1908483],
 'type': 'node',
 'visible': None}
{'created': {'changeset': '41096881',
             'timestamp': '2016-07-28T23:41:20Z',
             'uid': '326503',
             'user': 'wambag',
             'version': '5'},
 '

In [25]:
client = MongoClient("mongodb://localhost:27017")

db  = client.NewYorkOSMDB
db.drop
collection = db.NewYorkMap
#remove any stale data if its present
db.NewYorkMap.drop()
#insert the json Map
collection.insert_many(data)

print db.NewYorkMap.find_one()

{u'created': {u'changeset': u'41015803', u'version': u'4', u'uid': u'326503', u'timestamp': u'2016-07-25T17:17:46Z', u'user': u'wambag'}, u'pos': [40.6995927, -74.1868914], u'visible': None, u'_id': ObjectId('59898f46601fb725dc681c2e'), u'type': u'node', u'id': u'26769789'}


In [19]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'NewYorkOSMDB'), u'NewYorkMap')

Compute relative sizes

In [26]:
print 'The JSON File size is {} MB'.format(os.path.getsize(osm_file+'.json')/1.0e6) #1.0e6 is used to convert the bits to bytes

The JSON File size is 520.55966 MB


This is bit more to the original file size ie  511.26328 MB, due to the descriptive JSON format. Without doubt the entire Map is loaded

In [40]:
all = collection.find()
print 'There are totally {} records in the New York Map Collection'.format(all.count())

print 'There are {} unique contributors '.format(len(collection.distinct('created.user')))


There are totally 2271746 records in the New York Map Collection


There are 2136 unique contributors 


In [46]:
print "The are {} Nodes in this data set".format(collection.find({'type': 'node'}).count())
print "There are {} Ways in this data set".format(collection.find({'type': 'way'}).count())

The are 1925728 Nodes in this data set


There are 346018 Ways in this data set
